In [1]:
import pandas as pd
from thefuzz import fuzz

import warnings
warnings.filterwarnings("ignore")

## Readin and Initial Overview of Data

In [2]:
df = pd.read_excel('evictions_2023.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16657 entries, 0 to 16656
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   JP PRECINCT            16657 non-null  object        
 1   CaseNumber             16657 non-null  object        
 2   CaseFileDate           16657 non-null  datetime64[ns]
 3   CORP                   16657 non-null  object        
 4   Plaintiff              16657 non-null  object        
 5   PERSON_ALIAS_ID        16657 non-null  int64         
 6   PlaintiffPhone         16657 non-null  object        
 7   PlaintiffAddress       16657 non-null  object        
 8   Pl_City                16657 non-null  object        
 9   Pl_State               16657 non-null  object        
 10  Pl_Zip                 16657 non-null  object        
 11  DefendantAddress       16028 non-null  object        
 12  Unit Number            8116 non-null   object        
 13  C

In [4]:
og_data = df.shape[0]
print(f'Original Dataframe Length: {og_data}')

Original Dataframe Length: 16657


## Get Relevant Columns

In [5]:
for col in df.columns:
    
    print(col)

JP PRECINCT
CaseNumber
CaseFileDate
CORP
Plaintiff
PERSON_ALIAS_ID
PlaintiffPhone
PlaintiffAddress
Pl_City
Pl_State
Pl_Zip
DefendantAddress
Unit Number
CITY_NAME
STATE_CD
POSTAL_CD
Defendant
Def_Phone
CaseStatusDescription
JUDGMENT_DT
Judgment
Disposition
DispositionDate
DISP_AMT
CASE_TYP_DESC


In [6]:
# get relevant columns and rename for clarity
df = df[['CaseNumber',     # used to distinguish unique cases
         'CaseFileDate',   # used for case date
         'JUDGMENT_DT',    # used for date case was ruled on
         'CITY_NAME',      # used for city location
         'POSTAL_CD',      # used for zip code
         'Disposition']]   # used to determine if case was dismissed

df = df.rename(columns={'CaseNumber'   : 'case_number',
                        'CaseFileDate' : 'file_date',
                        'JUDGMENT_DT'  : 'judgement_date',
                        'CITY_NAME'    : 'city',
                        'POSTAL_CD'    : 'zip_code',
                        'Disposition'  : 'disposition'})

## Data Contains Duplicated Case Numbers

In [7]:
df_dupes = df[df.case_number.duplicated(keep=False)]

df_dupes

,case_number,file_date,judgement_date,city,zip_code,disposition
2,11E2303045,2023-08-16,2024-01-11,San Antonio,78214,Dismissed
3,11E2303045,2023-08-16,2024-01-11,San Antonio,78214,Dismissed
15,11E2304375,2023-11-03,2024-02-01,San Antonio,78227,Dismissed
16,11E2304375,2023-11-03,2024-02-01,San Antonio,78227,Judgment for Plaintiff (OCA)
17,11E2304393,2023-11-03,2024-04-11,SAN ANTONIO,78211,Default Judgments (OCA)
...,...,...,...,...,...,...
16053,41E2406012,2024-10-01,2024-10-29,San Antonio,78223,Judgment for Plaintiff (OCA)
16270,41E2406393,2024-10-21,2024-11-19,NaN,NaN,Default Judgments (OCA)
16271,41E2406393,2024-10-21,2024-11-19,NaN,NaN,Default Judgments (OCA)
16323,41E2406496,2024-10-24,2024-11-19,San Antonio,78219,Default Judgments (OCA)


In [8]:
# remove fully duplicated rows
df = df.drop_duplicates()

df

,case_number,file_date,judgement_date,city,zip_code,disposition
0,11E2102330,2021-12-13,2024-09-17,SAN ANTONIO,78221,Judgment for Plaintiff (OCA)
1,11E2301639,2023-05-10,2024-05-01,SAN ANTONIO,78227,Agreed Judgments (OCA)
2,11E2303045,2023-08-16,2024-01-11,San Antonio,78214,Dismissed
4,11E2303297,2023-08-30,2024-02-08,San Antonio,78251,Default Judgments (OCA)
5,11E2303925,2023-10-09,2024-01-11,San Antonio,78253-4301,Judgment for Plaintiff (OCA)
...,...,...,...,...,...,...
16652,41E2407195,2024-11-27,2024-12-13,NaN,NaN,Default Judgments (OCA)
16653,41E2407257,2024-12-03,2024-12-17,San Antonio,78223,Judgment for Plaintiff (OCA)
16654,41E2407268,2024-12-03,2024-12-19,San Antonio,78239,Default Judgments (OCA)
16655,41E2407309,2024-12-05,2024-12-19,San Antonio,78202,Default Judgments (OCA)


## After Removing Duplicate Rows There are Still Duplicate Case Numbers 
* Cases likely appear in the system for each stage of litigation

In [9]:
df[df.case_number == '41E2406496']

,case_number,file_date,judgement_date,city,zip_code,disposition
16323,41E2406496,2024-10-24,2024-11-19,San Antonio,78219,Default Judgments (OCA)
16324,41E2406496,2024-10-24,2024-11-19,San Antonio,78219,Dismissed for Want of Prosecution (OCA)


In [10]:
df.disposition.value_counts()

Default Judgments (OCA)                         10129
Judgment for Plaintiff (OCA)                     5700
Appealed                                          589
Dismissed for Want of Prosecution (OCA)            35
Take Nothing Judgment (OCA)                        28
Non-suited or Dismissed by Plaintiff (OCA)         23
Agreed Judgments (OCA)                             19
Dismissed                                          10
Trial/Hearing by Judge/Hearing Officer (OCA)        6
Trial by Jury (OCA)                                 3
Dismissed for Want of Jurisdiction (OCA)            1
Name: disposition, dtype: int64

## Removing Cases that Were Dismissed and Dropping Disposition Column

In [11]:
# add boolean column to identify rows with dismissed in disposition columns
df['dismissed'] = df.disposition.apply(lambda x : 'Dismissed' in x)

# get list of case numbers that were dismissed
dis_df = df[df.dismissed == True]

dismissed = list(set(dis_df.case_number.tolist()))

# remove rows containing dismissed data from dataframe and drop dismissed column
df = df[~df.case_number.isin(dismissed)]

# drop dismissed and disposition column
df = df.drop(columns = ['dismissed', 'disposition'])

df.head(10)

,case_number,file_date,judgement_date,city,zip_code
0,11E2102330,2021-12-13,2024-09-17,SAN ANTONIO,78221
1,11E2301639,2023-05-10,2024-05-01,SAN ANTONIO,78227
4,11E2303297,2023-08-30,2024-02-08,San Antonio,78251
5,11E2303925,2023-10-09,2024-01-11,San Antonio,78253-4301
6,11E2304048,2023-10-17,2024-01-04,San Antonio,78226
7,11E2304072,2023-10-18,2024-07-09,SAN ANTONIO,78221
8,11E2304114,2023-10-19,2024-01-11,SAN ANTONIO,78221
9,11E2304258,2023-10-26,2024-06-18,SAN ANTONIO,78214
10,11E2304278,2023-10-27,2024-01-04,SAN ANTONIO,78211
11,11E2304289,2023-10-30,2024-01-23,SAN ANTONIO,78221


In [12]:
re_dis = df.shape[0]
lost_rows = og_data - re_dis
print(f'Dataframe Length After Removing Dismissed Cases: {re_dis}')
print(f'Rows Dropped: {lost_rows}')

Dataframe Length After Removing Dismissed Cases: 16442
Rows Dropped: 215


## Getting Relevant City Values
* Used the fuzz to adjust for misspellings and abreviations of San Antonio
* Data includes surrounding areas of San Antonio it is unclear if it shold be included

In [13]:
# adjust city values for misspellings of san antonio, lowercase all values, and keep only San Antonio data
def fuzzy_sa(value):
    ''' takes in a pandas value and returns san antonio if string is a 90 or higher ratio to san antonio
        otherwise returns lower cased version of the original string'''
    
    value = str(value).lower()
    
    sa1 = 'san antonio'
    sa2 = 'sa'
    
    ratio1 = fuzz.ratio(sa1,value)
    ratio2 = fuzz.ratio(sa2,value)
    
    if (ratio1 >= 80) or (ratio2 >= 60):
        
        return 'san antonio'
    
    else:
        
        return value

df['city'] = df.city.apply(fuzzy_sa)

cities = list(set(df.city.tolist()))

cities.sort()

cities

['adkins',
 'alamo heights',
 'atascosa',
 'austin',
 'balcones heights',
 'boerne',
 'castle hills',
 'converse',
 'coverse',
 'elmendorf',
 'helotes',
 'kirby',
 'leon valley',
 'live oak',
 'marion',
 'nan',
 'new braunfels',
 'olmos park',
 'saint hedwig',
 'san antonio',
 'san antonio, texas',
 'sandy oaks',
 'schertz',
 'selma',
 'somerset',
 'st hedwig',
 'st. hedwig',
 'sweeny',
 'terrell hills',
 'universal',
 'universal city',
 'von army',
 'von ormy',
 'vonormy',
 'windcrest']

In [14]:
# remove cities not in list of acceptaple cities
cities = ['san antonio']

df = df[df.city.isin(cities)]

df.head(10)

,case_number,file_date,judgement_date,city,zip_code
0,11E2102330,2021-12-13,2024-09-17,san antonio,78221
1,11E2301639,2023-05-10,2024-05-01,san antonio,78227
4,11E2303297,2023-08-30,2024-02-08,san antonio,78251
5,11E2303925,2023-10-09,2024-01-11,san antonio,78253-4301
6,11E2304048,2023-10-17,2024-01-04,san antonio,78226
7,11E2304072,2023-10-18,2024-07-09,san antonio,78221
8,11E2304114,2023-10-19,2024-01-11,san antonio,78221
9,11E2304258,2023-10-26,2024-06-18,san antonio,78214
10,11E2304278,2023-10-27,2024-01-04,san antonio,78211
11,11E2304289,2023-10-30,2024-01-23,san antonio,78221


In [15]:
# drop city column
df = df.drop(columns = 'city')

In [16]:
re_cit = df.shape[0]
lost_rows = re_dis - re_cit
print(f'Dataframe Length After Removing Outside Cities: {re_cit}')
print(f'Rows Dropped: {lost_rows}')

Dataframe Length After Removing Outside Cities: 14963
Rows Dropped: 1479


## Normalizing Data in zip_code to 5 Digit Values 
* Data in some values is presented in 9 digit format while the majority is in 5 digit format


In [17]:
# get first five digits of values in zip_code column

df['zip_code'] = df.zip_code.apply(lambda x : str(x)[:5])

df.head(10)

,case_number,file_date,judgement_date,zip_code
0,11E2102330,2021-12-13,2024-09-17,78221
1,11E2301639,2023-05-10,2024-05-01,78227
4,11E2303297,2023-08-30,2024-02-08,78251
5,11E2303925,2023-10-09,2024-01-11,78253
6,11E2304048,2023-10-17,2024-01-04,78226
7,11E2304072,2023-10-18,2024-07-09,78221
8,11E2304114,2023-10-19,2024-01-11,78221
9,11E2304258,2023-10-26,2024-06-18,78214
10,11E2304278,2023-10-27,2024-01-04,78211
11,11E2304289,2023-10-30,2024-01-23,78221


## Removing Duplicate Rows
* All duplicate rows successfully removed

In [18]:
# drop duplicate rows
df = df.drop_duplicates()

non_dupe = df.shape[0]
dupes_dropped = re_cit - non_dupe

# test for duplicate case numbers

# list of case numbers
li = df.case_number.to_list()

# set of case numbers
se = set(li)

# list and let are the same length
verdict = len(li) == len(se)

print(f'All Duplicates Removed: {verdict}')

All Duplicates Removed: True


In [19]:
print(f'Dataframe Length After Dropping Duplicate Rows: {non_dupe}')
print(f'Rows Dropped: {dupes_dropped}')

Dataframe Length After Dropping Duplicate Rows: 14393
Rows Dropped: 570


## Final Breakdown
* Data contains only cases ruled on during 2024


In [20]:
min_file = str(df.file_date.min())[:4]
max_file = str(df.file_date.max())[:4]
min_judge = str(df.judgement_date.min())[:4]
max_judge = str(df.judgement_date.max())[:4]

rows_dropped = og_data - non_dupe

print(f'The original data contained {og_data}')
print(f'A total of {rows_dropped} rows were dropped during cleaning')
print(f'{non_dupe} remain after cleaning')

print()
print(f'Filing dates range betwen {min_file} and {max_file}')
print(f'Ruling dates range betwen {min_judge} and {max_judge}')

The original data contained 16657
A total of 2264 rows were dropped during cleaning
14393 remain after cleaning

Filing dates range betwen 2019 and 2024
Ruling dates range betwen 2024 and 2024


In [21]:
df.head(10)

,case_number,file_date,judgement_date,zip_code
0,11E2102330,2021-12-13,2024-09-17,78221
1,11E2301639,2023-05-10,2024-05-01,78227
4,11E2303297,2023-08-30,2024-02-08,78251
5,11E2303925,2023-10-09,2024-01-11,78253
6,11E2304048,2023-10-17,2024-01-04,78226
7,11E2304072,2023-10-18,2024-07-09,78221
8,11E2304114,2023-10-19,2024-01-11,78221
9,11E2304258,2023-10-26,2024-06-18,78214
10,11E2304278,2023-10-27,2024-01-04,78211
11,11E2304289,2023-10-30,2024-01-23,78221
